In [17]:
from PIL import Image
import json
import numpy as np

def load_data(image_files, json_files):
    images = []
    labels = []
    for img_file, json_file in zip(image_files, json_files):
        # Load image
        img = Image.open(img_file)
        img = img.convert('RGB')
        images.append(np.array(img))
        
        # Load JSON
        with open(json_file, 'r') as file:
            data = json.load(file)
        # Assuming JSON data is a list of coordinates [{'x': .., 'y': ..}, ...]
        labels.append(data)
    
    return np.array(images), labels


In [68]:
import os
import json
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def load_data(img_folder, json_folder):
    images = []
    labels = []
    for i in range(1, 101):  # Assuming you have 100 images and JSON files
        img_path = img_folder + f'/image_{i}.png' #os.path.join(img_folder, f'image_{i}.png')
        json_path = json_folder + f'/label_{i}.json' #os.path.join(json_folder, f'label_{i}.json')

        # Load image
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error loading image: {img_path}")
            continue  # Skip this image if it cannot be loaded
        image = cv2.resize(image, (224, 224))  # Resize image to match the CNN input
        images.append(image)

        # Load JSON data
        with open(json_path, 'r') as file:
            data = json.load(file)
            boxes = []
            for item in data['boxes']:
                if item['label'] == 'ball':
                    x = float(item['x']) / data['width']
                    y = float(item['y']) / data['height']
                    width = float(item['width']) / data['width']
                    height = float(item['height']) / data['height']
                    boxes.append([x, y, width, height])
            labels.append(boxes)

    # Normalize images
    images = np.array(images) / 255.0
    labels = np.array(labels)
    return images, labels

# Load dataset
img_folder = '../images'  # Update this to the correct path
json_folder = '../json_labeling'  # Update this to the correct path
images, labels = load_data(img_folder, json_folder)

# Split data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (100,) + inhomogeneous part.

In [47]:
import os
def load_images(image_dir):
    image_files = [os.path.join(image_dir, f) for f in sorted(os.listdir(image_dir)) if f.endswith(('.png', '.jpg', '.jpeg'))]
    images = []
    for img_file in image_files:
        img = Image.open(img_file).convert('RGB')  # Convert to RGB to ensure 3 color channels
        images.append(np.array(img))
    return images

In [50]:
import os
import json

def load_labels(json_dir):
    json_files = [os.path.join(json_dir, f) for f in sorted(os.listdir(json_dir)) if f.endswith('.json')]
    labels = []
    for json_file in json_files:
        with open(json_file, 'r') as file:
            data = json.load(file)
        # Access the 'boxes' key, which contains the list of objects
        bounding_boxes = []
        for obj in data['boxes']:
            # Ensure values are treated as floats since they appear as strings in the JSON
            x = float(obj['x'])
            y = float(obj['y'])
            width = float(obj['width'])
            height = float(obj['height'])
            bounding_boxes.append([x, y, x + width, y + height])
        labels.append(bounding_boxes)
    return labels

In [20]:
def convolve2d(image, kernel, stride=1, padding=1):
    # Adding padding to the input image
    image_padded = np.pad(image, [(padding, padding), (padding, padding), (0, 0)], mode='constant', constant_values=0)
    kernel_height, kernel_width = kernel.shape[0], kernel.shape[1]
    output_height = (image.shape[0] - kernel_height + 2 * padding) // stride + 1
    output_width = (image.shape[1] - kernel_width + 2 * padding) // stride + 1
    
    # Initialize the output feature maps
    output = np.zeros((output_height, output_width, image.shape[2]))
    
    # Perform convolution
    for z in range(image.shape[2]):  # loop over the channels
        for y in range(output_height):
            for x in range(output_width):
                output[y, x, z] = np.sum(
                    image_padded[y*stride:y*stride+kernel_height, x*stride:x*stride+kernel_width, z] * kernel[:, :, z])
    return output



In [21]:
def relu(feature_map):
    # Applying ReLU activation element-wise
    relu_out = np.maximum(0, feature_map)
    return relu_out


In [22]:
def max_pooling(feature_map, size=2, stride=2):
    # Initialize the output shape
    pool_out_height = (feature_map.shape[0] - size) // stride + 1
    pool_out_width = (feature_map.shape[1] - size) // stride + 1
    output = np.zeros((pool_out_height, pool_out_width, feature_map.shape[2]))
    
    for z in range(feature_map.shape[2]):
        for r in range(pool_out_height):
            for c in range(pool_out_width):
                output[r, c, z] = np.max(feature_map[r*stride:r*stride+size, c*stride:c*stride+size, z])
    return output


In [23]:
def mean_squared_error(predictions, targets):
    return ((predictions - targets) ** 2).mean()


In [24]:
def conv_backward(d_out, input, weight, bias):
    d_input = np.zeros_like(input)
    d_weight = np.zeros_like(weight)
    d_bias = np.zeros_like(bias)
    
    # Padding the input and input gradient for border cases
    padded_input = np.pad(input, pad_width=((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
    padded_d_input = np.pad(d_input, pad_width=((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
    
    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            region = padded_input[i:i+3, j:j+3]
            d_out_ij = d_out[i, j]
            d_weight += region * d_out_ij
            d_bias += d_out_ij
            padded_d_input[i:i+3, j:j+3] += weight * d_out_ij
    
    d_input = padded_d_input[1:-1, 1:-1]  # Remove padding
    return d_input, d_weight, d_bias


In [25]:
def forward_pass(input_image, weights, biases):
    # Example architecture: [Conv -> ReLU -> Pool] * 2 -> Flatten -> Dense (Fully Connected)
    # Conv Layer 1
    conv1 = convolve2d(input_image, weights['conv1'], stride=1, padding=1) + biases['conv1']
    relu1 = relu(conv1)
    pool1 = max_pooling(relu1, size=2, stride=2)
    
    # Conv Layer 2
    conv2 = convolve2d(pool1, weights['conv2'], stride=1, padding=1) + biases['conv2']
    relu2 = relu(conv2)
    pool2 = max_pooling(relu2, size=2, stride=2)
    
    # Flattening the output from the last pooling layer
    flat = pool2.reshape(-1)
    
    # Fully Connected Layer
    fc = np.dot(flat, weights['fc1']) + biases['fc1']
    output = relu(fc)  # Output layer activation could be softmax or another, depending on task
    
    return output


In [26]:
def conv_backward(d_out, input, weight, bias):
    d_input = np.zeros_like(input)
    d_weight = np.zeros_like(weight)
    d_bias = np.zeros_like(bias)
    
    # Padding the input and input gradient for border cases
    padded_input = np.pad(input, pad_width=((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
    padded_d_input = np.pad(d_input, pad_width=((1, 1), (1, 1), (0, 0)), mode='constant', constant_values=0)
    
    for i in range(input.shape[0]):
        for j in range(input.shape[1]):
            region = padded_input[i:i+3, j:j+3]
            d_out_ij = d_out[i, j]
            d_weight += region * d_out_ij
            d_bias += d_out_ij
            padded_d_input[i:i+3, j:j+3] += weight * d_out_ij
    
    d_input = padded_d_input[1:-1, 1:-1]  # Remove padding
    return d_input, d_weight, d_bias


In [27]:
def update_weights(weights, biases, d_weights, d_biases, learning_rate):
    weights -= learning_rate * d_weights
    biases -= learning_rate * d_biases
    return weights, biases


In [28]:
def backpropagation(y_pred, y_true, input, weights, biases):
    # Calculate initial gradient of loss function (Mean Squared Error in this case)
    d_loss = 2 * (y_pred - y_true)  # Derivative of MSE = 2 * (predicted - true)

    # Backpropagate through the fully connected layer
    d_fc_output = d_loss  # As there's no activation after final layer in this example
    d_fc_input = np.dot(d_fc_output, weights['fc'].T)
    d_fc_weights = np.dot(input['fc'].T, d_fc_output)
    d_fc_biases = d_fc_output.sum(axis=0)

    # Backpropagate ReLU activation
    d_relu = d_fc_input * (input['relu'] > 0)  # Gradient of ReLU is 1 for positive inputs, else 0

    # Backpropagate through the convolutional layer
    # Assuming conv_backward is defined to handle the specifics of conv layer backprop
    d_conv_input, d_conv_weights, d_conv_biases = conv_backward(d_relu, input['conv'], weights['conv'])

    # Package gradients in dictionaries to return
    gradients = {
        'weights': {
            'conv': d_conv_weights,
            'fc': d_fc_weights
        },
        'biases': {
            'conv': d_conv_biases,
            'fc': d_fc_biases
        }
    }

    return gradients


In [32]:
'''
def initialize_network():
    # This should return initialized weights and biases according to your network architecture
    weights = {
        'conv': np.random.randn(filter_size, filter_size, input_channels, num_filters),
        'fc': np.random.randn(num_features, num_outputs)
    }
    biases = {
        'conv': np.random.randn(num_filters),
        'fc': np.random.randn(num_outputs)
    }
    return weights, biases

# Load and preprocess data
data = load_images('./image_folder')  # Make sure this function is defined to load images from the directory
labels = load_labels('./json_folder')  # Make sure this function is defined to load labels from the directory

# Initialize network weights and biases
weights, biases = initialize_network()

# Training loop
learning_rate = 0.01  # A learning rate of 1 is usually too high; adjust based on your network's performance
num_epochs = 20

for epoch in range(num_epochs):
    for i in range(len(data)):
        # Forward pass
        outputs = forward_pass(data[i], weights, biases)  # Ensure this function correctly computes the forward pass
        
        # Calculate loss
        loss = mean_squared_error(outputs, labels[i])  # Ensure labels are correctly formatted for this loss calculation
        
        # Backward pass
        gradients = backpropagation(outputs, labels[i], data[i], weights, biases)  # Make sure this returns the necessary gradients
        
        # Update weights
        weights, biases = update_weights(weights, biases, gradients, learning_rate)  # Ensure updates are applied correctly
        
        if i % 100 == 0:
            print(f"Epoch {epoch}, Step {i}, Loss {loss:.4f}")
'''

'\ndef initialize_network():\n    # This should return initialized weights and biases according to your network architecture\n    weights = {\n        \'conv\': np.random.randn(filter_size, filter_size, input_channels, num_filters),\n        \'fc\': np.random.randn(num_features, num_outputs)\n    }\n    biases = {\n        \'conv\': np.random.randn(num_filters),\n        \'fc\': np.random.randn(num_outputs)\n    }\n    return weights, biases\n\n# Load and preprocess data\ndata = load_images(\'./image_folder\')  # Make sure this function is defined to load images from the directory\nlabels = load_labels(\'./json_folder\')  # Make sure this function is defined to load labels from the directory\n\n# Initialize network weights and biases\nweights, biases = initialize_network()\n\n# Training loop\nlearning_rate = 0.01  # A learning rate of 1 is usually too high; adjust based on your network\'s performance\nnum_epochs = 20\n\nfor epoch in range(num_epochs):\n    for i in range(len(data)):\n 

In [53]:
import numpy as np

def initialize_network():
    # Define network architecture parameters
    filter_size = 3  # example: 3x3 filters
    input_channels = 1  # example: grayscale images
    num_filters = 32  # example: 32 filters in the first conv layer
    num_features = 128  # example: 128 inputs to the fully connected layer
    num_outputs = 10  # example: 10 output classes

    # Initialize weights and biases
    weights = {
        'conv': np.random.randn(filter_size, filter_size, input_channels, num_filters) * np.sqrt(2. / (filter_size * filter_size * input_channels)),
        'fc': np.random.randn(num_features, num_outputs) * np.sqrt(2. / num_features)
    }
    biases = {
        'conv': np.zeros(num_filters),
        'fc': np.zeros(num_outputs)
    }
    return weights, biases


In [54]:
import numpy as np

def initialize_network(filter_size, input_channels, num_filters, num_features, num_outputs):
    # Initialize weights and biases with proper scaling
    weights = {
        'conv': np.random.randn(filter_size, filter_size, input_channels, num_filters) * np.sqrt(2. / (filter_size * filter_size * input_channels)),
        'fc': np.random.randn(num_features, num_outputs) * np.sqrt(2. / num_features)
    }
    biases = {
        'conv': np.zeros(num_filters),
        'fc': np.zeros(num_outputs)
    }
    return weights, biases


In [55]:
weights, biases = initialize_network(3, 1, 32, 128, 10)


In [56]:
import os

# Define the network architecture or load it if it's predefined
weights, biases = initialize_network()

for epoch in range(20):  # Number of epochs
    data = load_images("../images")  # Ensure this path is correct
    labels = load_labels("../json_labeling")  # Ensure this path is correct and data format matches expected input

    for i in range(len(data)):
        try:
            # Forward pass
            outputs = forward_pass(data[i], weights, biases)

            # Calculate loss
            loss = mean_squared_error(outputs, labels[i])

            # Backward pass
            gradients = backpropagation(outputs, data[i], labels[i], weights, biases)

            # Update weights
            learning_rate = 0.01  # A more typical value for learning rate
            weights, biases = update_weights(weights, biases, gradients, learning_rate)

            if i % 100 == 0:
                print(f"Epoch {epoch}, Step {i}, Loss {loss}")
        except Exception as e:
            print(f"An error occurred at epoch {epoch}, step {i}: {str(e)}")



TypeError: initialize_network() missing 5 required positional arguments: 'filter_size', 'input_channels', 'num_filters', 'num_features', and 'num_outputs'